In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import  OrdinalEncoder
from sklearn.linear_model import *
from sklearn.ensemble import ExtraTreesClassifier


In [2]:
train = pd.read_csv('./train.csv').drop(columns=['ID'])
test = pd.read_csv('./test.csv').drop(columns=['ID'])

In [3]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

### 대량 결측치 제거 + 평균값 대체

In [4]:
df_droped = X.drop(columns=['임신 시도 또는 마지막 임신 경과 연수', '배아 해동 경과일', '난자 해동 경과일'])
test_droped = test.drop(columns=['임신 시도 또는 마지막 임신 경과 연수', '배아 해동 경과일', '난자 해동 경과일'])
X = df_droped.fillna(df_droped.mean(numeric_only=True))
test = test_droped.fillna(df_droped.mean(numeric_only=True))
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256351 entries, 0 to 256350
Data columns (total 64 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   시술 시기 코드            256351 non-null  object 
 1   시술 당시 나이            256351 non-null  object 
 2   시술 유형               256351 non-null  object 
 3   특정 시술 유형            256349 non-null  object 
 4   배란 자극 여부            256351 non-null  int64  
 5   배란 유도 유형            256351 non-null  object 
 6   단일 배아 이식 여부         256351 non-null  float64
 7   착상 전 유전 검사 사용 여부    256351 non-null  float64
 8   착상 전 유전 진단 사용 여부    256351 non-null  float64
 9   남성 주 불임 원인          256351 non-null  int64  
 10  남성 부 불임 원인          256351 non-null  int64  
 11  여성 주 불임 원인          256351 non-null  int64  
 12  여성 부 불임 원인          256351 non-null  int64  
 13  부부 주 불임 원인          256351 non-null  int64  
 14  부부 부 불임 원인          256351 non-null  int64  
 15  불명확 불임 원인           256351 non-nul

In [5]:
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

In [6]:
# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

In [7]:
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

In [8]:
model = ExtraTreesClassifier(random_state=42)

model.fit(X_train_encoded, y)

ExtraTreesClassifier(random_state=42)

In [9]:
pred_proba = model.predict_proba(X_test_encoded)[:, 1]

In [10]:
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['probability'] = pred_proba

In [11]:
sample_submission.to_csv('./submit_2.csv', index=False)